In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys,gc
pd.set_option("max_columns",300)
pd.set_option("max_rows",300)

In [2]:
train = pd.read_csv("../input/train.csv")

In [3]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [4]:
st = pd.read_csv("../input/structures.csv")

In [5]:
st.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [6]:
train = train.merge(st, left_on=["molecule_name","atom_index_0"], right_on=["molecule_name","atom_index"], 
                    how="left").drop("atom_index",axis=1)

In [7]:
train.rename(columns={"atom":"atom_0","x":"x_0","y":"y_0","z":"z_0"},inplace=True)

In [8]:
train = train.merge(st, left_on=["molecule_name","atom_index_1"], right_on=["molecule_name","atom_index"], 
                    how="left").drop("atom_index",axis=1)

In [9]:
train.rename(columns={"atom":"atom_1","x":"x_1","y":"y_1","z":"z_1"},inplace=True)

In [10]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001


In [11]:
p0 = train[["x_0","y_0","z_0"]].values
p1 = train[["x_1","y_1","z_1"]].values
train["dist"] = np.linalg.norm(p0-p1, axis=1)

In [12]:
def rotM_test(p):
    # 回転行列を計算する
    ## X軸
    cos_x = p[:,2] / np.linalg.norm(p[:,[1,2]],axis=1)
    sin_x = p[:,1] / np.linalg.norm(p[:,[1,2]],axis=1)
    ## Y軸
    cos_y = p[:,0] / np.linalg.norm(p[:,[0,2]],axis=1)
    sin_y = p[:,2] / np.linalg.norm(p[:,[0,2]],axis=1)
    ## Z軸
    cos_z = p[:,1] / np.linalg.norm(p[:,[0,1]],axis=1)
    sin_z = p[:,0] / np.linalg.norm(p[:,[0,1]],axis=1)

    n_rows = len(p)
    # 物体座標系の 1->2->3 軸で回転させる
    Rx = np.array([[np.ones(n_rows), np.zeros(n_rows), np.zeros(n_rows)],
                   [np.zeros(n_rows), cos_x, sin_x],
                   [np.zeros(n_rows), -sin_x, cos_x]]).transpose([2,0,1])
    Ry = np.array([[cos_y, np.zeros(n_rows), -sin_y],
                   [np.zeros(n_rows), np.ones(n_rows), np.zeros(n_rows)],
                   [sin_y, np.zeros(n_rows), cos_y]]).transpose([2,0,1])
    Rz = np.array([[cos_z, sin_z, np.zeros(n_rows)],
                   [-sin_z, cos_z, np.zeros(n_rows)],
                   [np.zeros(n_rows), np.zeros(n_rows), np.ones(n_rows)]]).transpose([2,0,1])
    R = np.zeros([p.shape[0],p.shape[1],p.shape[1]])
    # 各行ごとに内積で回転行列を求める
    for ix in range(len(R)):
        R[ix,:,:] = Rz[ix].dot(Ry[ix]).dot(Rx[ix])
    return R

In [13]:
df = train.iloc[:1000]

In [14]:
vec = df[["x_1","y_1","z_1"]].values - df[["x_0","y_0","z_0"]].values

In [15]:
r_mat = rotM_test(vec)

/Users/kyo/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/Users/kyo/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [19]:
st_arr = st[["x","y","z"]].values

In [26]:
df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,1.091953
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,1.783120
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,1.783147
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,1.783157
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,1.091952


In [20]:
st_arr[0]

array([-0.01269814,  1.08580416,  0.008001  ])

In [24]:
np.dot(st_arr[0], r_mat[0])

array([0.00109179, 0.01418332, 1.0858147 ])

In [29]:
np.dot((st_arr[1]-st_arr[0]), r_mat[0])

array([-0.00226525, -0.01160806, -1.09188901])

## 一番上だけで試してみる

In [30]:
vec_xyz = (train[["x_1","y_1","z_1"]].values - train[["x_0","y_0","z_0"]].values)[0]

In [31]:
vec_xyz

array([-0.01484855,  1.09183548,  0.00602488])

In [32]:
r_mat0 = rotM_test(np.array([vec_xyz]))

In [33]:
r_mat0

array([[[-0.92654073,  0.37586785, -0.01567266],
        [-0.01260061,  0.0106302 ,  0.9998641 ],
        [ 0.37598337,  0.9266123 , -0.00511315]]])

In [35]:
np.dot(vec_xyz, r_mat0) - train[["x_0","y_0","z_0"]].values[0]

array([[1.14836986e-04, 1.76393821e-02, 1.08991289e+00]])

In [37]:
np.dot(vec_xyz, r_mat0)

array([[0.00226525, 0.01160806, 1.09188901]])

## もっとシンプル

In [379]:
def rotM(p):
    #p = np.array([0,0,1]) - p
    #print(p)
    # 回転行列を計算する
    ## X軸
    cos_x =  p[2] / np.linalg.norm(p)
    sin_x = np.linalg.norm(p[[0,1]]) / np.linalg.norm(p)#[[1,2]])
    #cos_x = np.sqrt(1-sin_x**2)
    #sin_x = np.sqrt(1-cos_x**2)
    ## Y軸
    #cos_y = p[2] / np.linalg.norm(p[[0,2]])
    #sin_y = p[0] / np.linalg.norm(p[[0,2]])
    ## Z軸
    cos_z = p[1] / np.linalg.norm(p[[0,1]])
    sin_z = p[0] / np.linalg.norm(p[[0,1]])

    n_rows = len(p)
    # 物体座標系の 1->2->3 軸で回転させる
    Rx = np.array([[1, 0, 0],
                   [0, cos_x, -sin_x],
                   [0, sin_x, cos_x]])
    #Ry = np.array([[cos_y, 0, sin_y],
    #               [0, 1, 0],
    #               [-sin_y, 0, cos_y]])
    Rz = np.array([[cos_z, -sin_z, 0],
                   [sin_z, cos_z, 0],
                   [0, 0,1]])
    # 各行ごとに内積で回転行列を求める
    R = Rx.dot(Rz)
    print(p)
    print(Rx.dot(Rz.dot(p)))
    return R

In [382]:
a = np.array([0.2,0.2,0.2])
#a = np.array([-1.4,9.5,0.5])
a = np.array([0.5,-0.5,0.2])

In [383]:
r_mat1 = rotM(a)

[ 0.5 -0.5  0.2]
[ 0.00000000e+00 -2.77555756e-17  7.34846923e-01]


In [385]:
r_mat1.dot(np.array([3,2,4]))

array([-3.53553391, -3.6565517 ,  1.76907593])

In [264]:
r_mat1[0].shape, a.shape

((3,), (3,))

In [194]:
np.dot(r_mat1,  a)

array([0.24142136, 0.24142136, 0.05857864])

In [187]:
np.linalg.norm(np.dot(a, r_mat1))

0.34641016151377535

In [188]:
np.linalg.norm(a)

0.3464101615137755